In [ ]:
import cv2
import time
import json
import requests
from collections import defaultdict
from ultralytics import YOLO
from ultralytics.trackers.byte_tracker import BYTETracker
import numpy as np

##############################################
# 1. YOLOv8 학습 모델 로드
##############################################
model = YOLO("/home/jetson/best.pt")   # 🔥 학습한 best.pt 경로 수정

##############################################
# 2. ByteTrack 설정
##############################################
tracker = BYTETracker(
    track_thresh=0.25,
    track_buffer=30,
    match_thresh=0.8,
    mot20=False
)

##############################################
# 3. ArUco 설정
##############################################
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
aruco_params = cv2.aruco.DetectorParameters()
aruco_detector = cv2.aruco.ArucoDetector(aruco_dict, aruco_params)

marker_to_point = {
    1: "alpha",
    2: "sector1", 3: "sector2", 4: "sector3",
    5: "sector4", 6: "sector5", 7: "sector6",
    8: "bravo",
    9: "sector7",
    10: "charlie",
    11: "sector8", 12: "sector9",
    13: "finish"
}

visited_points = set()

##############################################
# 4. Jetson 카메라 파이프라인
##############################################
pipeline = (
    "nvarguscamerasrc ! video/x-raw(memory:NVMM), width=640, height=480, format=NV12, framerate=30/1 ! "
    "nvvidconv ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink"
)

cap = cv2.VideoCapture(pipeline, cv2.CAP_GSTREAMER)
if not cap.isOpened():
    print("[ERROR] Camera open failed")
    exit()

##############################################
# 5. 서버 전송 함수
##############################################
def send_to_server(point, detected_objects):
    data = {
        "mission_code": "####",
        "points": [point],
        "detection": detected_objects
    }

    json_content = json.dumps(data, indent=2, ensure_ascii=False)
    files = {
        'file': (f"{point}.json", json_content, 'application/json')
    }

    try:
        print(f"[SEND] {point} → 서버 전송 중…")
        rsp = requests.post("http://58.229.150.23:5000/dashboard_json", files=files, timeout=10)
        print("[Server Response]", rsp.text)
    except Exception as e:
        print("[ERROR] 서버 전송 실패:", e)


##############################################
# 6. 메인 루프
##############################################
object_names = ['box','car','enemy','hazmat','missile','mortar','tank']

print("[START] YOLO + ArUco + ByteTrack 시스템 실행")

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    frame = cv2.flip(frame, -1)

    # -------------------------------------------------
    # (A) ArUco 포인트 감지
    # -------------------------------------------------
    corners, ids, _ = aruco_detector.detectMarkers(frame)

    if ids is not None:
        ids_list = [int(i[0]) for i in ids]

        for marker_id in ids_list:
            if marker_id in marker_to_point:
                point_name = marker_to_point[marker_id]

                if point_name not in visited_points:
                    visited_points.add(point_name)
                    print(f"[POINT] {point_name} 최초 통과 → 서버 전송")

                    # 객체 감지 내용 포함하여 전송
                    # 감지된 객체 개수 누적은 아래 YOLO 감지에서 계산됨
                    send_to_server(point_name, {"info": "passing point"})

        cv2.aruco.drawDetectedMarkers(frame, corners, ids)

    # -------------------------------------------------
    # (B) YOLO 객체 감지 + ByteTrack
    # -------------------------------------------------
    results = model.track(
        frame,
        tracker="bytetrack.yaml",
        persist=True,
        conf=0.45
    )

    detected_counts = defaultdict(int)

    if results[0].boxes.id is not None:
        ids = results[0].boxes.id.cpu().numpy()
        classes = results[0].boxes.cls.cpu().numpy()

        for cls in classes:
            class_name = object_names[int(cls)]
            detected_counts[class_name] += 1

    # -------------------------------------------------
    # (C) 객체 감지 결과가 있으면 서버로 전송
    # -------------------------------------------------
    if len(detected_counts) > 0:
        print("[DETECTION] 객체 감지:", dict(detected_counts))
        send_to_server("object-detect", dict(detected_counts))

    # -------------------------------------------------
    # (D) 화면 출력
    # -------------------------------------------------
    annotated = results[0].plot()
    cv2.imshow("YOLO + ArUco + Tracking", annotated)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

print("[END] 프로그램 종료")
